In [53]:
import numpy as np
import torch.nn as nn
from PIL import Image
import glob
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import pandas as pd
import math
import collections

N = 6283
D_in = 400
D_in2 = 20
H = 100
D_out = 62

x_list = []
x_data = np.zeros([N, D_in])
x_data2 = np.zeros([N, 1, D_in2, D_in2])
y_data = np.zeros([N, D_out])
y_data2 = np.zeros([N])

print("a")

#Reading images into numpy array. All images are stored as a row with 20*20 = 400 columns on x_data np array.

for img_id in range(N + 1):
    for filename in glob.glob("C:\\Users\\ahmetcan\\Documents\\SWE\\SWE 582\\Google Street View\\trainResized\\" 
                              + str(img_id) + ".bmp"):
        im=Image.open(filename)
        pix = np.array(im)
        
        if len(pix.shape) > 2:
            pix2 = pix.mean(2)
            pix3 = pix2.reshape((1, D_in))
            x_data2[img_id - 1, 0, :, :] = pix2
        else:
            pix3 = pix.reshape((1, D_in))
            x_data2[img_id - 1, 0, :, :] = pix
            
        x_list.append(pix3)
        x_data[img_id - 1] = pix3

print("b")

a
b


In [54]:
#The labels of the images are converted to integers.

def conv_lbl_to_int(a):
    if a.islower():
        return ord(a) - 61
    elif a.isnumeric():
        return int(a)
    else:
        return ord(a) - 55
    
df_labels = pd.read_csv('trainLabels.csv',header=0)

for index, row in df_labels.iterrows():
    if index < N:
        y_data2[index] = conv_lbl_to_int(row['Class'])

y_data2

array([ 49.,   8.,  29., ...,  25.,  23.,  27.])

In [57]:
x_data.shape

train_size = 5000

#The train size is determined and the whole training data is divided into training and test sets.

x_trn_data = x_data[: train_size, :]
x_tst_data = x_data[train_size: , :]

y_trn_data = y_data2[: train_size]
y_tst_data = y_data2[train_size:]

print(x_trn_data.shape)
print(x_tst_data.shape)

print(y_trn_data.shape)
print(y_tst_data.shape)

(5000, 400)
(1283, 400)
(5000,)
(1283,)


In [58]:
#K = 3

#Trying to find the best K value for KNN.

for K in range(1, 6):
    success = 0
    for i in range(N - train_size):
        nn = np.zeros([K])
        distances = np.sqrt(np.sum(np.square(x_trn_data - x_tst_data[i,:]), axis = 1))
        #print(i)
        #print(np.square(x_trn_data - x_tst_data[i,:]))
        knn_idx = np.argsort(distances)[:K]
        #print("knn_idx", knn_idx)
        for j in range(K):
            #print(y_trn_data[knn_idx[j]])
            nn[j] = y_trn_data[knn_idx[j]]
        unique, counts = np.unique(nn, return_counts=True)
        #print("counts: ",dict(zip(unique, counts)))
        #print("unique", unique)  
        #print("nn", nn)  
        #print("result")
        if np.amax(counts) == 1:
            guess = nn[0]
            #print(nn[0])
        else:
            guess = unique[np.argmax(counts)]
            #print()
        
        if guess == y_tst_data[i]:
            success = success + 1
    print("K: ", K)
    print("success: ", success)
    print("size", N - train_size)
    print("success rate: ", success / (N - train_size))
        
    #unique, counts = numpy.unique(a, return_counts=True)
    #dict(zip(unique, counts))
print("end")

K:  1
success:  507
size 1283
success rate:  0.3951675759937646
K:  2
success:  507
size 1283
success rate:  0.3951675759937646
K:  3
success:  507
size 1283
success rate:  0.3951675759937646
K:  4
success:  499
size 1283
success rate:  0.38893219017926733
K:  5
success:  489
size 1283
success rate:  0.38113795791114574
end
